In [ ]:
import os
import pickle
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm.auto import tqdm # to show progress bar
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose,BatchNormalization,Activation,AveragePooling2D,Concatenate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from keras.backend import set_session
from sklearn.metrics import jaccard_score

In [ ]:
train_contrail = np.load('/content/drive/MyDrive/train_contrail.npy')[1800:]
train_mask = np.load('/content/drive/MyDrive/train_mask.npy')[1800:]
valid_contrail= np.load('/content/drive/MyDrive/valid_contrail.npy')[500:]
valid_mask = np.load('/content/drive/MyDrive/valid_mask.npy')[500:]

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, UpSampling2D, Concatenate, Conv2DTranspose

# # Hyperparameters
# input_shape = (256, 256, 3)
# num_classes = 1  # Background and Contrail

# def build_encoder(input_tensor, num_layers):
#     x = input_tensor
#     for _ in range(num_layers):
#         x = Conv2D(64, (3, 3), padding='same')(x)
#         x = BatchNormalization()(x)
#         x = Activation('relu')(x)
#         x = Conv2D(64, (3, 3), padding='same')(x)
#         x = BatchNormalization()(x)
#         x = Activation('relu')(x)
#         x = AveragePooling2D(pool_size=(2, 2))(x)
#     return x

# def build_decoder(input_tensor, num_layers):
#     x = input_tensor
#     for _ in range(num_layers):
#         x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(x)
#         x = BatchNormalization()(x)
#         x = Activation('relu')(x)
#         x = Conv2DTranspose(64, (3, 3), padding='same')(x)
#         x = BatchNormalization()(x)
#         x = Activation('relu')(x)
#         x = UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
#     return x

# def build_deeplab_like(input_tensor):
#     # Encoder
#     x=build_encoder(input_tensor,80)
#     # ... add more encoder layers

#     # ASPP module
#     aspp = x  # Use the output of the last encoder layer
#     conv1x1 = Conv2D(32, (1, 1), padding='same', activation='relu')(aspp)
#     avg_pool = AveragePooling2D(pool_size=(2, 2))(aspp)
#     avg_pool_conv = Conv2D(32, (1, 1), padding='same', activation='relu')(avg_pool)
#     upsampled_avg_pool = UpSampling2D(size=(2, 2), interpolation='bilinear')(avg_pool_conv)

#     # Make sure the upsampled_avg_pool has the same shape as conv1x1
#     upsampled_avg_pool = Conv2D(32, (1, 1), padding='same', activation='relu')(upsampled_avg_pool)

#     concatenated = Concatenate()([conv1x1, upsampled_avg_pool])
#     aspp_output = Conv2D(32, (1, 1), padding='same', activation='relu')(concatenated)

#     # Decoder
#     x= build_decoder(input_tensor,80)
#     # ... add more decoder layers

#     return x  # Output of the decoder

# def build_deeplab(input_shape, num_classes):
#     inputs = Input(shape=input_shape)

#     # Encoder
#     x = Conv2D(32, (3, 3), padding='same')(inputs)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Conv2D(32, (3, 3), padding='same')(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     encoder_output = AveragePooling2D(pool_size=(2, 2))(x)
#     # ... add more encoder layers

#     # ASPP module
#     aspp = encoder_output
#     # ... ASPP operations

#     # Decoder
#     x = UpSampling2D(size=(2, 2), interpolation='bilinear')(aspp)
#     x = Conv2D(32, (3, 3), padding='same')(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Conv2D(32, (3, 3), padding='same')(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     # ... add more decoder layers

#     # Output layer
#     outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(x)

#     model = tf.keras.models.Model(inputs, outputs)
#     return model

# def dice_coefficient(y_true, y_pred):
#     smooth = 1.0
#     intersection = tf.reduce_sum(y_true * y_pred)
#     union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
#     return (2.0 * intersection + smooth) / (union + smooth)

# def dice_loss(y_true, y_pred):
#     return 1.0 - dice_coefficient(y_true, y_pred)



# # Instantiate and compile the model
# deeplab_model = build_deeplab(input_shape, num_classes)
# deeplab_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01), loss=dice_loss, metrics=[dice_coefficient])

# # Print model summary
# deeplab_model.summary()


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, UpSampling2D, Concatenate, Conv2DTranspose

# Hyperparameters
input_shape = (256, 256, 3)
num_classes = 1  # Background and Contrail
num_encoder_layers = 10  # Reduced number of encoder layers
num_decoder_layers = 10  # Reduced number of decoder layers

def build_encoder(input_tensor, num_layers):
    x = input_tensor
    for _ in range(num_layers):
        x = Conv2D(64, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(64, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        if _ % 2 == 0:  # Apply pooling every two layers
            x = AveragePooling2D(pool_size=(2, 2))(x)
    return x

def build_decoder(input_tensor, num_layers):
    x = input_tensor
    for _ in range(num_layers):
        x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(64, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        if _ % 2 == 0:  # Apply upsampling every two layers
            x = UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
    return x

def build_deeplab(input_shape, num_classes, num_encoder_layers, num_decoder_layers):
    inputs = Input(shape=input_shape)

    # Encoder
    encoder_output = build_encoder(inputs, num_encoder_layers)

    # ASPP module
    aspp = encoder_output
    # ... ASPP operations

    # Decoder
    decoder_output = build_decoder(aspp, num_decoder_layers)
    # ... add more decoder layers

    # Output layer
    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(decoder_output)

    model = tf.keras.models.Model(inputs, outputs)
    return model

def dice_coefficient(y_true, y_pred):
    smooth = 1.0
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return (2.0 * intersection + smooth) / (union + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)

# Instantiate and compile the model
deeplab_model = build_deeplab(input_shape, num_classes, num_encoder_layers, num_decoder_layers)
deeplab_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01), loss=dice_loss, metrics=[dice_coefficient])

# Print model summary
deeplab_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 64)     256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 256, 256, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 batch_normalization_1 (Batc  (None, 256, 256, 64)     256       
 hNormalization)                                             

In [ ]:
batch_size = 4
epochs = 10
train_steps_per_epoch = len(train_contrail) // batch_size
valid_steps_per_epoch = len(valid_contrail) // batch_size

history = deeplab_model.fit(
    x=train_contrail, y=train_mask,
    batch_size=batch_size,
    epochs=epochs,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=(valid_contrail, valid_mask),
    validation_steps=valid_steps_per_epoch
)

Epoch 1/10


ValueError: ignored

In [ ]:
print("shape of Valid contrail and train contrail is respectively", valid_contrail.shape,train_contrail.shape)
print("shape of Valid mask and train mask is respectively", valid_mask.shape,train_mask.shape)

shape of Valid contrail and train contrail is respectively (700, 256, 256, 3) (3000, 256, 256, 3)
shape of Valid mask and train mask is respectively (700, 256, 256, 1) (3000, 256, 256, 1)


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, UpSampling2D, Conv2DTranspose

# # Hyperparameters
# input_shape = (256, 256, 3)
# num_classes = 1  # Background and Contrail

# def build_deeplab(input_shape, num_classes):
#     inputs = Input(shape=input_shape)

#     # Encoder
#     x = Conv2D(64, (3, 3), padding='same')(inputs)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Conv2D(64, (3, 3), padding='same')(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     encoder_output = AveragePooling2D(pool_size=(2, 2))(x)

#     # ASPP module
#     aspp = encoder_output

#     # Decoder
#     x = UpSampling2D(size=(2, 2), interpolation='bilinear')(aspp)
#     x = Conv2D(64, (3, 3), padding='same')(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Conv2D(64, (3, 3), padding='same')(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     decoder_output = Conv2DTranspose(num_classes, (1, 1), activation='sigmoid')(x)

#     model = tf.keras.models.Model(inputs, decoder_output)
#     return model

# # Instantiate and compile the model
# deeplab_model = build_deeplab(input_shape, num_classes)
# deeplab_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# # Print model summary
# deeplab_model.summary()

# # Train the model
# batch_size = 4
# epochs = 10
# train_steps_per_epoch = len(train_contrail) // batch_size
# valid_steps_per_epoch = len(valid_contrail) // batch_size

# history = deeplab_model.fit(
#     x=train_contrail, y=train_mask,
#     batch_size=batch_size,
#     epochs=epochs,
#     steps_per_epoch=train_steps_per_epoch,
#     validation_data=(valid_contrail, valid_mask),
#     validation_steps=valid_steps_per_epoch
# )


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 64)     256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 256, 256, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 batch_normalization_1 (Batc  (None, 256, 256, 64)     256       
 hNormalization)                                             

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, UpSampling2D, Conv2DTranspose, Concatenate

# Hyperparameters
input_shape = (256, 256, 3)
num_classes = 1  # Background and Contrail

def build_deeplab(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Encoder
    x = Conv2D(64, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    encoder_output = AveragePooling2D(pool_size=(2, 2))(x)

    # ASPP module
    dilations = [1, 3, 6, 12]  # Atrous rates
    aspp_layers = []
    for dilation_rate in dilations:
        aspp_layer = Conv2D(64, (3, 3), padding='same', dilation_rate=dilation_rate)(encoder_output)
        aspp_layer = BatchNormalization()(aspp_layer)
        aspp_layer = Activation('relu')(aspp_layer)
        aspp_layers.append(aspp_layer)

    aspp_concat = Concatenate()(aspp_layers)
    aspp_output = Conv2D(64, (1, 1), padding='same', activation='relu')(aspp_concat)

    # Decoder
    x = UpSampling2D(size=(2, 2), interpolation='bilinear')(aspp_output)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    decoder_output = Conv2DTranspose(num_classes, (1, 1), activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs, decoder_output)
    return model

# Instantiate and compile the model
deeplab_model = build_deeplab(input_shape, num_classes)
deeplab_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
deeplab_model.summary()

# Train the model
batch_size = 4
epochs = 10
train_steps_per_epoch = len(train_contrail) // batch_size
valid_steps_per_epoch = len(valid_contrail) // batch_size

history = deeplab_model.fit(
    x=train_contrail, y=train_mask,
    batch_size=batch_size,
    epochs=epochs,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=(valid_contrail, valid_mask),
    validation_steps=valid_steps_per_epoch
)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

KeyboardInterrupt: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, UpSampling2D, Conv2DTranspose, Concatenate, DepthwiseConv2D

# Hyperparameters
input_shape = (256, 256, 3)
num_classes = 1  # Background and Contrail
num_encoder_layers = 6  # Increase this
num_decoder_layers = 6  # Increase this

def build_encoder(input_tensor, num_layers):
    x = input_tensor
    for _ in range(num_layers):
        x = Conv2D(128, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(128, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = AveragePooling2D(pool_size=(2, 2))(x)
    return x

def build_decoder(input_tensor, num_layers):
    x = input_tensor
    for _ in range(num_layers):
        x = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(128, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
    return x

def build_deeplab(input_shape, num_classes, num_encoder_layers, num_decoder_layers):
    inputs = Input(shape=input_shape)

    # Encoder
    encoder_output = build_encoder(inputs, num_encoder_layers)

    # ASPP module
    dilations = [1, 3, 6, 12]  # Atrous rates
    aspp_layers = []
    for dilation_rate in dilations:
        aspp_layer = DepthwiseConv2D((3, 3), padding='same', dilation_rate=dilation_rate)(encoder_output)
        aspp_layer = Conv2D(128, (1, 1), padding='same')(aspp_layer)
        aspp_layer = BatchNormalization()(aspp_layer)
        aspp_layer = Activation('relu')(aspp_layer)
        aspp_layers.append(aspp_layer)

    aspp_concat = Concatenate()(aspp_layers)
    aspp_output = Conv2D(128, (1, 1), padding='same', activation='relu')(aspp_concat)

    # Decoder
    decoder_output = build_decoder(aspp_output, num_decoder_layers)
    decoder_output = Conv2DTranspose(num_classes, (1, 1), activation='sigmoid')(decoder_output)

    model = tf.keras.models.Model(inputs, decoder_output)
    return model

# Instantiate and compile the model
deeplab_model = build_deeplab(input_shape, num_classes, num_encoder_layers, num_decoder_layers)
deeplab_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
deeplab_model.summary()

# Train the model
batch_size = 4
epochs = 10
train_steps_per_epoch = len(train_contrail) // batch_size
valid_steps_per_epoch = len(valid_contrail) // batch_size

history = deeplab_model.fit(
    x=train_contrail, y=train_mask,
    batch_size=batch_size,
    epochs=epochs,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=(valid_contrail, valid_mask),
    validation_steps=valid_steps_per_epoch
)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 12  3584        ['input_1[0][0]']                
                                8)                                                                
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 12  512        ['conv2d[0][0]']                 
 alization)                     8)                                                            

In [ ]:
model_save_path = 'deeplab_model.h5'
deeplab_model.save(model_save_path)